In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df_tramites_proc_X = pd.read_parquet('./data/tramita/tramites_autoconsumo.parquet')

In [3]:
df_tramites_proc_X.head()

,id_exp,dni,nif,fecha_alta_exp,fecha_registro_exp,codine_provincia,codine_municipio,codine,municipio,provincia,...,siaci,consejeria_proc,org_instructor_proc,es_telematica,desc_tramite,fecha_tramite,num_tramite,cod_procedimiento,orden_tramite,es_inicial
45875,1127895,1787973,<NA>,2022-02-09 09:07:17,2021-12-23,13,064,13064,Poblete,Ciudad Real,...,SLPT,Consejeria de desarrollo sostenible,Direccion general de transicion energetica,True,Registro de solicitud,2021-12-23 00:00:00,0,884.0,0,False
45876,1127895,1787973,<NA>,2022-02-09 09:07:17,2021-12-23,13,064,13064,Poblete,Ciudad Real,...,SLPT,Consejeria de desarrollo sostenible,Direccion general de transicion energetica,True,1-1-Presentación de la solicitud,2022-02-09 09:07:17,1,884.0,1,True
45877,1127895,1787973,<NA>,2022-02-09 09:07:17,2021-12-23,13,064,13064,Poblete,Ciudad Real,...,SLPT,Consejeria de desarrollo sostenible,Direccion general de transicion energetica,True,10-1-Requerimiento documentación,2022-10-14 09:44:23,10,884.0,2,False
45878,1127895,1787973,<NA>,2022-02-09 09:07:17,2021-12-23,13,064,13064,Poblete,Ciudad Real,...,SLPT,Consejeria de desarrollo sostenible,Direccion general de transicion energetica,True,30-1-Evaluación de Solicitud,2022-10-27 08:04:51,30,884.0,3,False
45879,1127895,1787973,<NA>,2022-02-09 09:07:17,2021-12-23,13,064,13064,Poblete,Ciudad Real,...,SLPT,Consejeria de desarrollo sostenible,Direccion general de transicion energetica,True,42-1-Propuesta Resolución Estimatoria de Conce...,2022-10-27 08:14:55,42,884.0,4,False


In [4]:
columns_estados = ['id_exp','desc_tramite','fecha_tramite']
df_tramites_proc_X_simple = df_tramites_proc_X[columns_estados]
df_tramites_proc_X_simple = df_tramites_proc_X_simple.rename(columns={'fecha_tramite': 'fecha'})
#df_tramites_proc_X_simple['fecha'] = df_tramites_proc_X_simple['fecha'].dt.date
df_tramites_proc_X_simple.head()

,id_exp,desc_tramite,fecha
45875,1127895,Registro de solicitud,2021-12-23 00:00:00
45876,1127895,1-1-Presentación de la solicitud,2022-02-09 09:07:17
45877,1127895,10-1-Requerimiento documentación,2022-10-14 09:44:23
45878,1127895,30-1-Evaluación de Solicitud,2022-10-27 08:04:51
45879,1127895,42-1-Propuesta Resolución Estimatoria de Conce...,2022-10-27 08:14:55


In [5]:
#asignamos fecha como nueva columna, quitándonos la hora
# df_estados_proc_X = df_estados_proc_X.assign(fecha=df_estados_proc_X['fecha_tramite'].dt.date)
# df_estados_proc_X.drop('fecha_tramite', axis = 1, inplace = True)
# df_estados_proc_X.head()
# Here are the possible ways to add a new column to a DataFrame that is calculated from another original column of the same DataFrame:

# Assign: df['new_column'] = df['original_column'] * 2
# Apply: df['new_column'] = df.apply(lambda x: x['original_column'] * 2, axis=1)
# Map: df['new_column'] = df['original_column'].map(lambda x: x * 2)
# Assign with conditional: df['new_column'] = np.where(df['original_column'] > 0, df['original_column'] * 2, 0)
# Using pandas' built-in functions: df['new_column'] = df['original_column'].rolling(window=3).mean()
# Using pandas' groupby: df.groupby('group_column')['original_column'].sum().reset_index(name='new_column')

# Número de expedientes en estado x por fecha, forward fill

In [6]:
df = df_tramites_proc_X_simple

## quitar duplicados

In [7]:
duplicates = df[df.duplicated(subset=['id_exp', 'fecha', 'desc_tramite'], keep=False)]
duplicates

,id_exp,desc_tramite,fecha
6496786,1239794,50-1-Resolución de Concesión,2024-07-18 07:11:53
6496787,1239794,50-1-Resolución de Concesión,2024-07-18 07:11:53
7234997,1171327,51-1-Resolución desestimatoria - Incumple Requ...,2023-10-23 13:03:36
7234998,1171327,51-1-Resolución desestimatoria - Incumple Requ...,2023-10-23 13:03:36


In [8]:
# drop duplicates
df = df.drop_duplicates(
    subset=['id_exp', 'fecha', 'desc_tramite'], 
    keep='last'
)

## quitar 15500 expedientes con finales "presentacion solicitud"

In [9]:
final_states = df.sort_values(['id_exp', 'fecha']).groupby('id_exp').last().reset_index()
# si se filtra sort_values('fecha') sale mal, hay algo que falla al ordenar
final_states

,id_exp,desc_tramite,fecha
0,1083865,287-1-Seguimiento,2023-07-04 07:48:09
1,1083888,287-1-Seguimiento,2023-05-04 09:14:05
2,1083890,70-1-Presentación Documentación Justificativa,2024-04-05 18:12:05
3,1083903,287-1-Seguimiento,2023-04-03 13:51:39
4,1083907,287-1-Seguimiento,2023-04-03 13:54:44
...,...,...,...
25840,1736598,1-1-Presentación de la solicitud,2024-01-18 12:17:29
25841,1738774,1-1-Presentación de la solicitud,2024-01-24 08:48:14
25842,1853399,1-1-Presentación de la solicitud,2024-02-22 12:55:09
25843,1861916,1-1-Presentación de la solicitud,2024-03-06 08:09:56


In [10]:
final_states[final_states['id_exp']==1084489]

,id_exp,desc_tramite,fecha
23,1084489,287-1-Seguimiento,2023-05-09 10:39:18


In [11]:
final_states_grp = final_states.groupby('desc_tramite').size()
final_states_grp

desc_tramite
1-1-Presentación de la solicitud                                   15152
10-1-Requerimiento documentación                                     397
10-2-Requerimiento documentación                                      92
100-1-Fin de procedimiento                                           682
152-1-Propuesta de Resolucion Desistimiento                           23
278-1-Resolución de Pérdida de Derecho al Cobro Total                  3
283-1-Propuesta Resolución de Pérdida de Derecho al Cobro Total        1
286-1-Presentación Solicitud Desistimiento                             1
287-1-Seguimiento                                                    851
30-1-Evaluación de Solicitud                                           6
309-1-Lista de Reserva Provisional por Insuficiencia de Credito        2
41-1-Propuesta Resolución Desestimatoria - Incumple Requisitos        21
42-1-Propuesta Resolución Estimatoria de Concesión                   949
50-1-Resolución de Concesión          

In [12]:
id_exp_to_remove = final_states.loc[(final_states['desc_tramite'] =='1-1-Presentación de la solicitud'), "id_exp"].unique()
id_exp_to_remove

array([1088250, 1094269, 1094633, ..., 1738774, 1853399, 1861916],
      dtype=uint32)

In [13]:
len(id_exp_to_remove)

15152

In [14]:
# Filter out rows with those `id_exp` values
filtered_df = df[~df["id_exp"].isin(id_exp_to_remove)]

In [15]:
len(filtered_df.id_exp.unique())

10693

## fill forward

In [18]:
filtered_df

,id_exp,desc_tramite,fecha
45875,1127895,Registro de solicitud,2021-12-23 00:00:00
45876,1127895,1-1-Presentación de la solicitud,2022-02-09 09:07:17
45877,1127895,10-1-Requerimiento documentación,2022-10-14 09:44:23
45878,1127895,30-1-Evaluación de Solicitud,2022-10-27 08:04:51
45879,1127895,42-1-Propuesta Resolución Estimatoria de Conce...,2022-10-27 08:14:55
...,...,...,...
9122628,1602703,1-1-Presentación de la solicitud,2023-07-24 07:48:53
9122629,1602703,10-1-Requerimiento documentación,2024-06-13 16:59:50
9122630,1602703,30-1-Evaluación de Solicitud,2024-07-12 09:47:49
9122631,1602703,42-1-Propuesta Resolución Estimatoria de Conce...,2024-07-12 09:49:15


In [25]:
# se mantiene la fecha con hh:mm porque es importante, si se deja solo el dia, puede haber 1 dia con 2 estados, y los dos se harán 
# fill forward hasta el siguiente, y si es el último, continúa
df = (
    filtered_df.reset_index(drop=True) # Reset index to access columns
    .sort_values(['id_exp', 'fecha'])  # Ensure proper sorting
)
df

,id_exp,desc_tramite,fecha
28514,1083865,Registro de solicitud,2021-12-22 00:00:00
28515,1083865,1-1-Presentación de la solicitud,2022-02-04 12:38:10
28516,1083865,10-1-Requerimiento documentación,2022-04-19 08:16:55
28517,1083865,30-1-Evaluación de Solicitud,2022-05-04 12:01:23
28518,1083865,42-1-Propuesta Resolución Estimatoria de Conce...,2022-06-08 08:33:19
...,...,...,...
29449,1956447,1-1-Presentación de la solicitud,2024-06-21 08:54:18
29450,1956447,10-1-Requerimiento documentación,2024-06-21 14:23:12
29451,1956447,30-1-Evaluación de Solicitud,2024-06-26 10:45:24
29452,1956447,42-1-Propuesta Resolución Estimatoria de Conce...,2024-06-26 10:46:48


In [23]:
df.loc[df.index.get_level_values(0) == 1084489]

desc_tramite
id_exp  fecha                                                                 
1084489 2021-12-22 00:00:00                              Registro de solicitud
        2022-02-04 14:05:05                   1-1-Presentación de la solicitud
        2022-06-10 13:18:27                       30-1-Evaluación de Solicitud
        2022-06-10 13:19:29  42-1-Propuesta Resolución Estimatoria de Conce...
        2022-08-10 07:39:14                       50-1-Resolución de Concesión
        2023-01-25 11:08:53      70-1-Presentación Documentación Justificativa
        2023-04-24 11:58:35                   30-2-Evaluación de justificación
        2023-04-24 12:00:49                                    82-1-Pago Total
        2023-05-09 10:39:18                                  287-1-Seguimiento

In [27]:
# Step 1: Create a complete date range for each id_exp
expanded = (
    df[["id_exp", "fecha"]]
    .groupby("id_exp", as_index=False)
    .agg(min_fecha=("fecha", "min"), max_fecha=("fecha", "max"))
    .apply(
        lambda row: pd.DataFrame(
            {
                "fecha": pd.date_range(
                    row["min_fecha"].date(),
                    row["max_fecha"].date(),
                    freq="D"
                ),
                "id_exp": row["id_exp"]
            }
        ),
        axis=1
    )
    .explode(ignore_index=True)
)

In [29]:
expanded.info()

<class 'pandas.core.series.Series'>
RangeIndex: 7758578 entries, 0 to 7758577
Series name: None
Non-Null Count  Dtype 
--------------  ----- 
21386 non-null  object
dtypes: object(1)
memory usage: 59.2+ MB


In [ ]:
result.to_csv('./data/tramita/full/forwardfill_total.csv', encoding='utf-8',  sep=';', index = True, index_label='fecha')

In [ ]:
column_states = ['Registro de solicitud',
                 '1-1-Presentación de la solicitud',
                #'287-1-Seguimiento',
                 '42-1-Propuesta Resolución Estimatoria de Concesión',
                ]
df_toplot = result[column_states]

In [ ]:
# Plot
df_toplot.plot(kind="area", stacked=False, figsize=(12, 6), legend=dict(loc='upper right'))
plt.title("Acumulación estados en el tiempo")
plt.xlabel("Fecha")
plt.ylabel("Numero de expedientes")
plt.legend(title="Estados")
plt.show()

# Número de cambios por dia


In [ ]:
# # tabla con el número total de cambios de estado , cada día, una columna por estado
# df_tramitesX_estados = df_tramitesX_estados.sort_values(by='fecha')
# # no hay valores para todas las fechas, habría que rellenar las fechas vacías.
# df_pivot_estados = df_tramitesX_estados.pivot_table(index='fecha', columns='num_tramite', aggfunc='size', fill_value=0)
# df_pivot_estados.tail()